# Slab subduction

In [145]:
# %%
import numpy as np
import os
import math
import underworld3

import plot


from underworld3 import petsc_gen_xdmf

In [146]:
outputPath = 'output/'

# Make output directory if necessary.
from mpi4py import MPI
if MPI.COMM_WORLD.rank==0:
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)

In [147]:
ppcell = 5

In [148]:
#Basic direct solver parameters from Firedrake docs, for ref

#mumps_solver_parameters = {
#    "mat_type": "aij",
#    "snes_type": "ksponly",
#    "ksp_type": "preonly",
#    "pc_type": "lu",
#    "pc_factor_mat_solver_type": "mumps",
#}

In [149]:
from petsc4py import PETSc
import underworld3 as uw
from underworld3.systems import Stokes

options = PETSc.Options()
# options["help"] = None
# options["pc_type"]  = "svd"
#options["ksp_rtol"] =  1.0e-6
#options["ksp_atol"] =  1.0e-6
#options["ksp_monitor"] = None
# options["snes_type"]  = "fas"
#options["snes_type"]="ksponly"
options["snes_converged_reason"] = None
options["snes_monitor_short"] = None
# options["snes_view"]=None
# options["snes_test_jacobian"] = None
# options["snes_rtol"] = 1.0e-2  # set this low to force single SNES it. 
options["snes_max_it"] = 3

options["mat_type"]="aij"

options["ksp_type"]="preonly"
options["pc_type"] = "lu"
options["pc_factor_mat_solver_type"] = "mumps"

sys = PETSc.Sys()
sys.pushErrorHandler("traceback")


In [150]:
n_els = 32
dim =2
boxLength = 4.0
boxHeight = 1.0

## Mesh stuff

In [151]:
mesh = uw.mesh.Box(elementRes=(    4*n_els,n_els), 
                    minCoords =(       0.,)*dim, 
                    maxCoords =(boxLength,boxHeight),
                    simplex=False )
u_degree = 1
stokes = Stokes(mesh, u_degree=u_degree )

## Swarm

In [152]:
# Create swarm
swarm  = uw.swarm.Swarm(mesh)
# Add variable for material
materialVariable      = swarm.add_variable(name="materialVariable",      num_components=1, dtype=PETSc.IntType)
dummyVariable      = swarm.add_variable(name="dummyVariable",      num_components=1, dtype=PETSc.IntType)

# Note that `ppcell` specifies particles per cell per dim.
swarm.populate(ppcell=ppcell)

In [153]:
#%%
# Add some randomness to the particle distribution
import numpy as np
np.random.seed(0)

with swarm.access(swarm.particle_coordinates):
    factor = 0.5*boxLength/n_els/ppcell
    swarm.particle_coordinates.data[:] += factor*np.random.rand(*swarm.particle_coordinates.data.shape)

## setup the material distribution

In [154]:
import matplotlib.path as mpltPath

In [155]:


# initialise the 'materialVariable' data to represent two different materials. 
upperMantleIndex = 0
lowerMantleIndex = 1
upperSlabIndex   = 2
lowerSlabIndex   = 3
coreSlabIndex    = 4

# Initial material layout has a flat lying slab with at 15\degree perturbation
lowerMantleY   = 0.4
slabLowerShape = np.array([ (1.2,0.925 ), (3.25,0.925 ), (3.20,0.900), (1.2,0.900), (1.02,0.825), (1.02,0.850) ])
slabCoreShape  = np.array([ (1.2,0.975 ), (3.35,0.975 ), (3.25,0.925), (1.2,0.925), (1.02,0.850), (1.02,0.900) ])
slabUpperShape = np.array([ (1.2,1.000 ), (3.40,1.000 ), (3.35,0.975), (1.2,0.975), (1.02,0.900), (1.02,0.925) ])

#slabLower = fn.shape.Polygon( slabLowerShape )

In [156]:
slabLower  =mpltPath.Path(slabLowerShape)
slabCore  =mpltPath.Path(slabCoreShape)
slabUpper  =mpltPath.Path(slabUpperShape)

In [157]:
with swarm.access(swarm.particle_coordinates, materialVariable):
    
    #if swarm.particle_coordinates.data[:][1] < lowerMantleY:
    materialVariable.data[:] = upperMantleIndex
    materialVariable.data[swarm.particle_coordinates.data[:,1] < lowerMantleY]     = lowerMantleIndex
    materialVariable.data[slabLower.contains_points(swarm.particle_coordinates.data[:])] = lowerSlabIndex
    materialVariable.data[slabCore.contains_points(swarm.particle_coordinates.data[:])] = coreSlabIndex
    materialVariable.data[slabUpper.contains_points(swarm.particle_coordinates.data[:])] = upperSlabIndex
    

## Rheology

In [158]:
from sympy import Piecewise, ceiling, Abs, Min, sqrt, eye

In [159]:
upperMantleViscosity =    1.0
lowerMantleViscosity =  50.0

slabViscosity        =  100.0
coreViscosity        =  250.0


sr = stokes.strainrate
# not sure if the following is needed as div_u should be zero
sr -= (stokes.div_u/mesh.dim)*eye(mesh.dim)
# second invariant of strain rate
inv2 = sr[0,0]**2 + sr[0,1]**2 + sr[1,0]**2 + sr[1,1]**2
inv2 = 1/2.*inv2
strainRate_2ndInvariant = sqrt(inv2)


cohesion = 0.06
vonMises = 0.5 * cohesion / (strainRate_2ndInvariant+1.0e-18)


# The upper slab viscosity is the minimum of the 'slabViscosity' or the 'vonMises' 
slabYieldvisc =  Min(vonMises, slabViscosity)



In [160]:
#start with solve on a linear viscosity
stokes.viscosity = Piecewise( (upperMantleViscosity, Abs(materialVariable.fn - upperMantleIndex) < 0.5),
                 (lowerMantleViscosity, Abs(materialVariable.fn -  lowerMantleIndex) < 0.5),
                 (slabViscosity, Abs(materialVariable.fn - upperSlabIndex) < 0.5),
                 (slabViscosity , Abs(materialVariable.fn - lowerSlabIndex) < 0.5),
                 (coreViscosity, Abs(materialVariable.fn - coreSlabIndex) < 0.5), 
                            ( lowerMantleViscosity,                                True ))





In [161]:


mantleDensity = 0.0
slabDensity   = 1.0 

density = Piecewise( (mantleDensity, Abs(materialVariable.fn - upperMantleIndex) < 0.5),
                 (mantleDensity, Abs(materialVariable.fn -  lowerMantleIndex) < 0.5),
                 (slabDensity, Abs(materialVariable.fn - upperSlabIndex) < 0.5),
                 (slabDensity , Abs(materialVariable.fn - lowerSlabIndex) < 0.5),
                 (slabDensity, Abs(materialVariable.fn - coreSlabIndex) < 0.5),
                    ( mantleDensity,                                True ))



stokes.bodyforce = -density*mesh.N.j



## boundary conditions

In [162]:
#free slip
bnds = mesh.boundary
stokes.add_dirichlet_bc( (0.,0.), [bnds.TOP,  bnds.BOTTOM], 1)  # top/bottom: function, boundaries, components 
stokes.add_dirichlet_bc( (0.,0.), [bnds.LEFT, bnds.RIGHT ], 0  )  # left/right: function, boundaries, components


## Initial Solve

In [163]:
stokes.solve(zero_init_guess=False)

In [164]:
stokes.viscosity = Piecewise( (upperMantleViscosity, Abs(materialVariable.fn - upperMantleIndex) < 0.5),
                 (lowerMantleViscosity, Abs(materialVariable.fn -  lowerMantleIndex) < 0.5),
                 (slabYieldvisc, Abs(materialVariable.fn - upperSlabIndex) < 0.5),
                 (slabYieldvisc , Abs(materialVariable.fn - lowerSlabIndex) < 0.5),
                 (coreViscosity, Abs(materialVariable.fn - coreSlabIndex) < 0.5), 
                            ( lowerMantleViscosity,                                True ))


## plot 

In [165]:
#figs = plot.Plot(rulers=True)
#with swarm.access(),mesh.access():
#        figs.swarm_points(swarm, materialVariable.data, pointsize=5, colourmap="coolwarm", colourbar=False)
#        figs.vector_arrows(mesh, stokes.u.data)
#        
#outputFilename = os.path.join(outputPath,"slab.png")
#figs.image(outputFilename)

## Main loop

In [166]:
step = 0
max_steps = 50


#timing setup
#viewer.getTimestep()
#viewer.setTimestep(1)


while step<max_steps:
    stokes.solve(zero_init_guess=False)
    
    #advect the particles
    dt = stokes.dt()
    with swarm.access():
        vel_on_particles = uw.function.evaluate(stokes.u.fn,swarm.particle_coordinates.data)

    with swarm.access(swarm.particle_coordinates):
        swarm.particle_coordinates.data[:]+=dt*vel_on_particles
        
    #viz for serial case  
    #with swarm.access(),mesh.access():
    #    figs = plot.Plot(rulers=True)
    #    figs.swarm_points(swarm, materialVariable.data, pointsize=4, colourmap="coolwarm", colourbar=False)
    #    figs.vector_arrows(mesh, stokes.u.data)
    #    # fig.nodes(mesh,matMeshVar.data,colourmap="blue green", pointsize=6, pointtype=4)
    #    outputFilename = os.path.join(outputPath,f"slab_yield_{str(step).zfill(4)}.png")
    #    figs.image(outputFilename)
    
   
    
    #viz for parallel case - write the hdf5s/xdmfs - veolocity and pressure
    if step%2==0:
    
        fname = f"./{outputPath}{'step_'}{step:02d}.h5"
        xfname = f"./{outputPath}{'step_'}{step:02d}.xdmf"
        viewer = PETSc.ViewerHDF5().createHDF5(fname, mode=PETSc.Viewer.Mode.WRITE,  comm=PETSc.COMM_WORLD)
        viewer(mesh.dm)
        viewer(stokes.u._gvec) #add velocity
        viewer(stokes.p._gvec) #add pressure
        viewer.destroy() 
        petsc_gen_xdmf.generateXdmf(fname, xfname)
    step +=1



    #viewer.setTimestep(step)
        


AttributeError: 'NoneType' object has no attribute 'mesh'

## issues

At many resolutions, I'm getting the following error message, when I try to evaluate the velocity on the particles. 

```
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-166-c06b8ae04b7c> in <module>
     14     dt = stokes.dt()
     15     with swarm.access():
---> 16         vel_on_particles = uw.function.evaluate(stokes.u.fn,swarm.particle_coordinates.data)
     17 
     18     with swarm.access(swarm.particle_coordinates):

/tmp/uw3/underworld3/function/_function.pyx in underworld3.function._function.evaluate()

/tmp/uw3/underworld3/function/_function.pyx in underworld3.function._function.evaluate.get_var_fns()

/tmp/uw3/underworld3/function/_function.pyx in underworld3.function._function.evaluate.get_var_fns()

/tmp/uw3/underworld3/function/_function.pyx in underworld3.function._function.evaluate.get_var_fns()

AttributeError: 'NoneType' object has no attribute 'mesh'


```